In [1]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import pprint
from bson.objectid import ObjectId
import pyperclip as clip
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools as it
# from datasu import auc
# import pixiedust
# clip.copy(json.dumps(m1['msg']))
import nbimporter
from common_functions import *
from nbimporter import NotebookLoader
loader = NotebookLoader()
from common_functions_message import *
Message = loader.load_module("common_functions_message").Message

pd.options.mode.chained_assignment = None  # default='warn'

Importing Jupyter notebook from common_functions.ipynb
Importing Jupyter notebook from common_functions_message.ipynb
Importing Jupyter notebook from common_functions_message.ipynb


### funcs

### load data

In [2]:
# MONGO_HOST = "ec2-52-23-187-115.compute-1.amazonaws.com"
MONGO_DB = "marketpulzz"
# server = SSHTunnelForwarder(
#     MONGO_HOST,
#     ssh_username='ubuntu',
#     ssh_pkey="/Users/sasha/.ssh/mmuze.pem",
# #     ssh_private_key_password="secret",
#     remote_bind_address=('127.0.0.1', 27017),
#     local_bind_address=('127.0.0.1', 63328),
#     set_keepalive = 5,
# )

# server.start()
# client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]

In [3]:
client_local = MongoClient('127.0.0.1', 27017)
db_local = client_local[MONGO_DB]

In [4]:
messages = list(db_local.genie_conversation_messages_emb.find({}))

In [8]:
def get_message_for_master_tag(mt):   
    mess = db_local.genie_conversation_messages.find_one({"_id": mt['conversation_message_id']})    
    fields = Message(mess).get_fields_for_vector(fixed_length=None, fld_name_suffix=None)
    return fields
    
master_tags_cursor = db_local.genie_master_tags.find({"intents.0":{'$exists':True}})
data = list(it.chain(*map(get_message_for_master_tag, master_tags_cursor)))
texts = list(set(it.chain(*map(lambda t: t[1] if isinstance(t[1], list) else [t[1]]  ,data)))) + ['*']

### embed messages

#### BERT

In [9]:
#bert-serving-start -model_dir /Users/sasha/dev/mmuze/_models/BERT/uncased_L-12_H-768_A-12/ -num_worker=1 -max_batch_size 16 -max_seq_len 200 -pooling_strategy REDUCE_MEAN -pooling_layer -2 
bert_metadata = {            
    'version': '1.0',
    'encoder': 'BERT',
    'model': 'uncased_L-12_H-768_A-12',    
    'max_seq_len': 200,
    'pooling_strategy': 'REDUCE_MEAN', 
    'pooling_layer': '-5',
#     'dimensions': emb_bert[0].shape[0]
}

print(f"bert-serving-start \
-model_dir /Users/sasha/dev/mmuze/_models/BERT/{bert_metadata['model']} \
-num_worker=1 \
-max_batch_size 16 \
-max_seq_len {bert_metadata['max_seq_len']} \
-pooling_strategy {bert_metadata['pooling_strategy']} \
-pooling_layer {bert_metadata['pooling_layer']}")

bert-serving-start -model_dir /Users/sasha/dev/mmuze/_models/BERT/uncased_L-12_H-768_A-12 -num_worker=1 -max_batch_size 16 -max_seq_len 200 -pooling_strategy REDUCE_MEAN -pooling_layer -5


In [10]:
from bert_serving.client import BertClient
bc = BertClient()

In [11]:
%%time
emb_bert = bc.encode(texts)

CPU times: user 149 ms, sys: 42.4 ms, total: 191 ms
Wall time: 12min 25s


In [13]:
def create_emb_cached_value(text, vector, metadata):
    metadata['dimensions']: len(vector),
    emb_cached = {
        'text': text,
        'vector': vector,    
        
        'metadata': metadata,        
    }
    return emb_cached

In [14]:
bert_emb_cache = list(map(lambda t: create_emb_cached_value(t[0],t[1].tolist(),bert_metadata) ,zip(texts, emb_bert)))

In [15]:
for m in bert_emb_cache:
    db_local.embeddings_cache.save(m, check_keys=False)

/usr/local/opt/pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  


In [16]:
len(bert_emb_cache)

2818

#### USE

In [17]:
# https://stackoverflow.com/questions/48092772/add-operation-to-graph-without-with-as-clause
import tensorflow_hub as hub
graph = tf.Graph()
cm = graph.as_default()   
cm.__enter__()
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url, trainable=True)

session = tf.Session(graph=graph)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
input1 = tf.placeholder(tf.string, shape=(None))
emb = embed(input1)


def encode_USE(texts):   
    emb1 = session.run([emb], feed_dict={ input1: texts })
    return emb1

#warm up
_ = encode_USE(["sasha", "miko"])

INFO:tensorflow:Using /var/folders/xn/dh4vn2gx4x36vh0x051kpw1m0000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [18]:
%%time
emb_USE = encode_USE(texts)[0]

CPU times: user 10min 47s, sys: 1min 22s, total: 12min 10s
Wall time: 2min 1s


In [19]:
use_metadata = {
    'version': '1.0',
    'encoder': 'USE',
    'model': module_url,    
}

In [20]:
use_emb_cache = list(map(lambda t: create_emb_cached_value(t[0],t[1].tolist(),use_metadata) ,zip(texts, emb_USE)))

In [21]:
for m in use_emb_cache:
    db_local.embeddings_cache.save(m, check_keys=False)

/usr/local/opt/pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  


In [113]:
z = get_embedding_from_cache('', use_metadata)